In [586]:
import pandas as pd
import numpy as np
from bokeh.plotting import figure, output_file, show

In [608]:
train_df = pd.read_csv("train.csv")
df2 = pd.DataFrame([[45, 100],[90,10],[90,10],[80,15],[95,10],[90,15],[75,10],[95,20],[40,90],[60,20]],columns=['x','y'])
train_df.dropna(inplace=True,axis=0,how='any')
new_train_df = train_df.append(df2)
#train_df.y.fillna(train_df.y.max(),inplace=True)
Y_train = train_df.y
X = train_df.x
m = Y_train.shape[0]
new_Y_train = new_train_df.y
new_X = new_train_df.x
new_m = new_Y_train.shape[0]
print("Y shape is : " , m)
train_df.head()

Y shape is :  699


,x,y
0,24.0,21.549452
1,50.0,47.464463
2,15.0,17.218656
3,38.0,36.586398
4,87.0,87.288984


In [588]:
#Plotting data
output_file("plot.html")
p = figure()
p.sizing_mode = 'scale_width'
p.circle(train_df.x,train_df.y,size=10,color='red',alpha=0.5)
show(p)

In [609]:
def computeCost(X,y,theta):
    size = y.shape[0]
    J = (1/(2*size)) * np.sum((np.matmul(X,theta) - y)**2)
    return J

In [610]:
def gradientDescent(X,y,theta,alpha,num_iter):
    print("initial theta shape is : ", theta)
    j = []
    #print("X contains null : " , np.isnan(Y).any())
    for i in range(num_iter):
        error = (np.dot(X,theta) - y)
        #print(np.isnan(error).any())
        theta = theta - ((alpha)*(np.dot(error,X)/y.shape[0]))
        j.append(computeCost(X,y,theta))
    print("theta is : ", theta)
        
    print("Cost is : " , computeCost(X,y,theta))
    return theta,j

In [611]:
X = np.transpose(X)
new_X = np.transpose(new_X)

In [612]:
X = np.c_[np.ones(m),X]
new_X = np.c_[np.ones(new_m),new_X]

In [613]:
print(X.shape)
print(np.sum(X))
print(np.isnan(Y_train).any())

(699, 2)
35659.0
False


In [614]:
theta = np.ones(X.shape[1])
new_theta = np.ones(new_X.shape[1])
iterations = 100000
alphas = [0.0005]
print(theta.shape)

(2,)


In [615]:
print("old : " ,computeCost(X,Y_train,theta))
print("new : " ,computeCost(new_X,new_Y_train,new_theta))

old :  4.511264121372125
new :  38.14298112953331


In [616]:
new_J_dic = {}
new_theta_dic = {}
J_dic = {}
theta_dic = {}
for i,alpha in enumerate(alphas):
    theta = np.ones(X.shape[1])
    theta,J = gradientDescent(X,Y_train,theta,alpha,iterations)
    J_dic[alpha] = J
    theta_dic[alpha] = theta
    
for i,alpha in enumerate(alphas):
    new_theta = np.ones(X.shape[1])
    new_theta,new_J = gradientDescent(new_X,new_Y_train,new_theta,alpha,iterations)
    new_J_dic[alpha] = new_J
    new_theta_dic[alpha] = new_theta

initial theta shape is :  [1. 1.]
theta is :  [-0.10726148  1.00065632]
Cost is :  3.9338763667458303
initial theta shape is :  [1. 1.]
theta is :  [1.03462365 0.96512935]
Cost is :  36.1492102891318


In [597]:
#Plotting J against iterations data
output_file("Jplot.html")
p1 = figure()
p1.sizing_mode = 'scale_width'
for color, alpha in zip(['red','green','blue','yellow','purple'],alphas):
    p1.line(list(range(1,iterations+1)),J_dic[alpha],line_width=2,color=color,
            alpha=0.8,muted_color=color, muted_alpha=0.2,legend=str(alpha))
p1.xaxis.axis_label = "Iterations"
p1.yaxis.axis_label = "J"
p1.legend.location = "top_right"
p1.legend.click_policy="mute"
show(p1)

In [617]:
test_df = pd.read_csv("test.csv")
test_df.loc[test_df.y.isnull()]
X_test = np.transpose(test_df.x)
m_test = test_df.y.shape[0]
X_test = np.c_[np.ones(m_test),X_test]
predict = np.dot(X_test,theta_dic[0.0005])
correct = [(i,j) for i, j in zip(test_df.y.values,predict)]
print(correct)

[(79.77515201, 76.94327532718559), (23.17727887, 20.906521283129866), (25.60926156, 21.907177605345147), (17.85738813, 19.905864960914585), (41.84986439, 35.91636611635908), (9.805234876, 14.902583349838183), (58.87465933, 61.93343049395637), (97.61793701, 94.95508912706063), (18.39512747, 19.905864960914585), (8.746747654, 4.896020127685377), (2.811415826, 3.895363805470096), (17.09537241, 18.905208638699303), (95.14907176, 95.95574544927592), (61.38800663, 61.93343049395637), (40.24701716, 35.91636611635908), (14.82248589, 14.902583349838183), (66.95806869, 64.93539946060221), (16.63507984, 13.901927027622904), (90.65513736, 86.9498385493384), (77.22982636, 68.93802474946334), (92.11906278, 88.95115119376895), (46.91387709, 50.92621094958829), (89.82634442, 88.95115119376895), (21.71380347, 26.91045921642155), (97.41206981, 96.9564017714912), (57.01631363, 57.930805205095254), (78.31056542, 78.94458797161614), (19.1315097, 20.906521283129866), (93.03483388, 92.95377648263008), (26.59

In [618]:
def slope_intercept(x,y,tht_dic):
    slope = round(tht_dic[0.0005][1],2)
    b = np.mean(y) - np.dot(np.mean(x),slope)
    b = round(tht_dic[0.0005][0],2)
    return slope,b

In [619]:
slope,b = slope_intercept(X, Y_train,theta_dic)
new_slope, new_b = slope_intercept(new_X,new_Y_train,new_theta_dic)

In [620]:
reg_line = [np.dot(slope,x) + b for x in train_df.x]
new_reg_line = [np.dot(new_slope,x) + new_b for x in new_train_df.x]

In [624]:
#Plotting data
output_file("plot2.html")
p2 = figure()
p2.sizing_mode = 'scale_width'
p2.circle(train_df.x,train_df.y,size=10,color='red',alpha=0.5)
p2.line(train_df.x,reg_line,line_width=5)
p2.line(new_train_df.x,new_reg_line,color='green',line_width=5)
show(p2)